# Desarrollo de un Super chat espectacular

## Paso 1: Instalación de dependencias
- OpenAI
- dotenv
- gradio

In [1]:
!pip install openai dotenv gradio

  Using cached numpy-2.3.3-cp313-cp313-win_amd64.whl.metadata (60 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
   ---------------------------------------- 0.0/964.9 kB ? eta -:--:--
   --------------------- ------------------ 524.3/964.9 kB 7.2 MB/s eta 0:00:01
   ---------------------------------------- 964.9/964.9 kB 3.1 MB/s  0:00:00
   ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
   ---------- ----------------------------- 0.5/2.0 MB 2.4 MB/s eta 0:00:01
   --------------------- ------------------ 1.0/2.0 MB 2.4 MB/s eta 0:00:01
   -------------------------------- ------- 1.6/2.0 MB 2.4 MB/s eta 0:00:01
   ---------------------------------------- 2.0/2.0 MB 2.3 MB/s  0:00:00
   ---------------------------------------- 0.0/63.5 MB ? eta -:--:--
   ---------------------------------------- 0.5/63.5 MB 3.1 MB/s eta 0:00:21
    --------------------------------------- 1.0/63.5

In [ ]:
!pip install transformers

## Paso 2. Cargar variables de entorno

In [3]:
from dotenv import load_dotenv
load_dotenv(override=True)

True